# 第 12 章：RNN 综合实战项目

> 将所学知识应用到实际项目中

---

## 项目列表

1. **情感分析** - BiLSTM 文本分类
2. **文本生成** - 字符级语言模型
3. **序列标注** - BiLSTM-CRF 命名实体识别
4. **机器翻译** - Seq2Seq + Attention
5. **股票预测** - LSTM 时间序列

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

---

## 项目一：情感分析

### 1.1 模型定义

In [ ]:
class SentimentClassifier(nn.Module):
    """
    基于 BiLSTM 的情感分类器
    
    结构: Embedding → BiLSTM → Attention → FC → Softmax
    """
    
    def __init__(self, vocab_size, embed_dim, hidden_size, num_classes, 
                 num_layers=2, dropout=0.3, pretrained_embeddings=None):
        super().__init__()
        
        # 词嵌入
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        if pretrained_embeddings is not None:
            self.embedding.weight.data.copy_(pretrained_embeddings)
        
        # 双向 LSTM
        self.lstm = nn.LSTM(
            embed_dim, hidden_size, num_layers,
            bidirectional=True, batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # 注意力层
        self.attention = nn.Linear(hidden_size * 2, 1)
        
        # 分类层
        self.fc = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, num_classes)
        )
    
    def forward(self, x, lengths=None):
        """
        x: (batch, seq_len) 词索引
        """
        # 词嵌入
        embedded = self.embedding(x)  # (batch, seq, embed)
        
        # BiLSTM
        output, _ = self.lstm(embedded)  # (batch, seq, hidden*2)
        
        # 注意力
        attn_weights = F.softmax(self.attention(output).squeeze(-1), dim=1)
        context = torch.bmm(attn_weights.unsqueeze(1), output).squeeze(1)
        
        # 分类
        logits = self.fc(context)
        return logits, attn_weights


# 示例
model = SentimentClassifier(
    vocab_size=10000, embed_dim=128, hidden_size=256, num_classes=2
)

# 模拟输入
x = torch.randint(0, 10000, (4, 50))
logits, attn = model(x)

print(f"输出形状: {logits.shape}")
print(f"注意力形状: {attn.shape}")

---

## 项目二：名字生成

### 2.1 字符级 RNN 生成名字

In [ ]:
class NameGenerator(nn.Module):
    """
    字符级名字生成器
    
    可以根据类别（如国家）生成对应风格的名字
    """
    
    def __init__(self, n_categories, vocab_size, embed_dim, hidden_size):
        super().__init__()
        
        self.hidden_size = hidden_size
        
        # 类别嵌入
        self.category_embedding = nn.Embedding(n_categories, embed_dim)
        
        # 字符嵌入
        self.char_embedding = nn.Embedding(vocab_size, embed_dim)
        
        # GRU (输入 = 类别 + 字符)
        self.gru = nn.GRU(embed_dim * 2, hidden_size, batch_first=True)
        
        # 输出层
        self.fc = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, category, chars, hidden=None):
        """
        category: (batch,) 类别索引
        chars: (batch, seq_len) 字符索引
        """
        batch_size, seq_len = chars.shape
        
        # 类别嵌入（扩展到所有时间步）
        cat_emb = self.category_embedding(category)  # (batch, embed)
        cat_emb = cat_emb.unsqueeze(1).expand(-1, seq_len, -1)  # (batch, seq, embed)
        
        # 字符嵌入
        char_emb = self.char_embedding(chars)  # (batch, seq, embed)
        
        # 拼接
        combined = torch.cat([cat_emb, char_emb], dim=2)  # (batch, seq, embed*2)
        
        # GRU
        output, hidden = self.gru(combined, hidden)
        
        # 预测下一个字符
        logits = self.fc(output)
        
        return logits, hidden
    
    def generate(self, category, start_char, max_len=20, temperature=1.0):
        """
        生成名字
        """
        self.eval()
        generated = [start_char]
        hidden = None
        
        with torch.no_grad():
            for _ in range(max_len):
                char_tensor = torch.tensor([[generated[-1]]])
                cat_tensor = torch.tensor([category])
                
                logits, hidden = self.forward(cat_tensor, char_tensor, hidden)
                probs = F.softmax(logits[0, 0] / temperature, dim=0)
                next_char = torch.multinomial(probs, 1).item()
                
                if next_char == 0:  # EOS
                    break
                generated.append(next_char)
        
        return generated


# 示例
gen = NameGenerator(n_categories=5, vocab_size=30, embed_dim=64, hidden_size=128)
print(f"模型参数: {sum(p.numel() for p in gen.parameters()):,}")

---

## 项目三：简单对话机器人

### 3.1 Seq2Seq 对话模型

In [ ]:
class ChatBot(nn.Module):
    """
    简单的 Seq2Seq 对话机器人
    """
    
    def __init__(self, vocab_size, embed_dim, hidden_size, num_layers=2):
        super().__init__()
        
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        
        # 共享词嵌入
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # 编码器
        self.encoder = nn.LSTM(embed_dim, hidden_size, num_layers,
                              batch_first=True, bidirectional=True)
        
        # 解码器
        self.decoder = nn.LSTM(embed_dim + hidden_size * 2, hidden_size, num_layers,
                              batch_first=True)
        
        # 注意力
        self.attention = nn.Linear(hidden_size * 3, 1)
        
        # 输出层
        self.fc = nn.Linear(hidden_size * 3, vocab_size)
    
    def forward(self, src, trg):
        """
        前向传播（训练时使用 teacher forcing）
        """
        batch_size = src.size(0)
        trg_len = trg.size(1)
        
        # 编码
        src_embedded = self.embedding(src)
        encoder_outputs, (hidden, cell) = self.encoder(src_embedded)
        
        # 调整 hidden 维度（双向 → 单向）
        hidden = hidden.view(self.encoder.num_layers, 2, batch_size, -1)
        hidden = hidden[:, 0, :, :] + hidden[:, 1, :, :]  # 相加
        cell = cell.view(self.encoder.num_layers, 2, batch_size, -1)
        cell = cell[:, 0, :, :] + cell[:, 1, :, :]
        
        # 解码
        outputs = []
        trg_embedded = self.embedding(trg)
        
        for t in range(trg_len):
            # 注意力
            h_expanded = hidden[-1].unsqueeze(1).expand(-1, encoder_outputs.size(1), -1)
            attn_input = torch.cat([h_expanded, encoder_outputs], dim=2)
            attn_weights = F.softmax(self.attention(attn_input).squeeze(-1), dim=1)
            context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)
            
            # 解码器输入
            dec_input = torch.cat([trg_embedded[:, t:t+1, :], context.unsqueeze(1)], dim=2)
            output, (hidden, cell) = self.decoder(dec_input, (hidden, cell))
            
            # 预测
            pred = self.fc(torch.cat([output.squeeze(1), context], dim=1))
            outputs.append(pred)
        
        return torch.stack(outputs, dim=1)  # (batch, trg_len, vocab)


# 示例
chatbot = ChatBot(vocab_size=5000, embed_dim=128, hidden_size=256)
print(f"ChatBot 参数量: {sum(p.numel() for p in chatbot.parameters()):,}")

---

## 本章总结

### 项目清单

| 项目 | 技术要点 | 难度 |
|------|----------|------|
| 情感分析 | BiLSTM + Attention | ⭐⭐ |
| 名字生成 | 条件语言模型 | ⭐⭐ |
| 对话机器人 | Seq2Seq + Attention | ⭐⭐⭐ |
| 机器翻译 | Seq2Seq + Beam Search | ⭐⭐⭐⭐ |
| 语音识别 | CTC + BiLSTM | ⭐⭐⭐⭐ |

### 后续学习

- **Transformer**: 下一个模块 `4.3_transformer/`
- **预训练模型**: BERT, GPT 等
- **大语言模型**: LLM 微调和应用

---

## 恭喜完成 RNN 模块！

你已经掌握了：
- RNN/LSTM/GRU 的原理和实现
- BPTT 反向传播算法
- 梯度消失问题和解决方案
- 双向和多层 RNN
- 语言模型和文本生成
- Seq2Seq 和注意力机制
- 时间序列预测

下一步：学习 Transformer，这是现代 NLP 的基础！

👉 [4.3_transformer/](../4.3_transformer/)